In [2]:
import os
import sys

os.chdir("C:/dataanalytics/python")
os.curdir

#Configure the environment . Set this up to the directory where spark is installed
if 'SPARK_HOME' not in os.environ:
    os.environ['SPARK_HOME'] = 'C:\\spark'
    
#create a variable for our root path
SPARK_HOME = os.environ['SPARK_HOME']

#Add the following paths to the system path. Please check your installation
#to make sure that these zip files actually exists. The names might change as
#versions change
sys.path.insert(0,os.path.join(SPARK_HOME,"python"))
sys.path.insert(0,os.path.join(SPARK_HOME,"python","lib"))
sys.path.insert(0,os.path.join(SPARK_HOME,"python","lib","py4j-0.10.6-src.zip"))
sys.path.insert(0,os.path.join(SPARK_HOME,"python","lib","pyspark.zip"))
 
#Initialize a spark context
from pyspark import SparkContext
from pyspark import SparkConf

#optionally configure spark
conf = SparkConf()
conf.setAppName("Jeffwiz")

#Initalize spark context onl runs once
sc = SparkContext('local', conf=conf)


In [3]:
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.regression import LinearRegressionWithSGD
from pyspark.mllib.classification import LogisticRegressionWithLBFGS
from pyspark.mllib.classification import LogisticRegressionModel
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.feature import HashingTF, IDF
from pyspark.mllib.classification import SVMWithSGD
from pyspark.mllib.classification import SVMModel
from pyspark.mllib.classification import NaiveBayes
from pyspark.mllib.clustering import KMeans
from pyspark.mllib.clustering import BisectingKMeans
from pyspark.mllib.classification import StreamingLogisticRegressionWithSGD
from pyspark.mllib.clustering import GaussianMixture
from pyspark.mllib.clustering import PowerIterationClustering
from pyspark.mllib.clustering import StreamingKMeans
from pyspark.mllib.clustering import LDA
#one for from pyspark.mllib.evaluation BinaryClassificationMetrics
from pyspark.mllib.feature import Normalizer
from pyspark.mllib.feature import StandardScaler
from pyspark.mllib.feature import Word2Vec
from pyspark.mllib.feature import ChiSqSelector
from pyspark.mllib.feature import ElementwiseProduct
from pyspark.mllib.fpm import PrefixSpan
from pyspark.mllib.linalg import Matrix
from pyspark.mllib.linalg import SparseMatrix
from pyspark.mllib.linalg import QRDecomposition
#for distributed matrixes check version
from pyspark.mllib.random import RandomRDDs
from pyspark.mllib.recommendation import MatrixFactorizationModel
from pyspark.mllib.regression import RidgeRegressionModel
from pyspark.mllib.regression import LassoModel
from pyspark.mllib.regression import IsotonicRegression
from pyspark.mllib.stat import Statistics
from pyspark.mllib.tree import DecisionTree
from pyspark.mllib.tree import RandomForest
from pyspark.mllib.tree import GradientBoostedTrees
from pyspark.mllib.util import JavaLoader
from pyspark.mllib.util import LinearDataGenerator
from pyspark.mllib.util import Loader
from pyspark.mllib.util import MLUtils







In [11]:
svmdata = sc.textFile(r'C:\spark\data\mllib\sample_svm_data.txt')
rows = svmdata.map(lambda line:line.split(" "))
for row in rows.take(rows.count()):print(row[0],'\t',row[1],'\t',row[2], '\t', row[5])

1 	 0 	 2.52078447201548 	 0
0 	 2.857738033247042 	 0 	 0
0 	 2.857738033247042 	 0 	 0
1 	 0 	 0 	 0
1 	 2.857738033247042 	 0 	 0
0 	 2.857738033247042 	 0 	 0
1 	 0 	 0 	 0
1 	 0 	 0 	 0
0 	 2.857738033247042 	 0 	 0
0 	 2.857738033247042 	 0 	 0
1 	 2.857738033247042 	 0 	 0
1 	 2.857738033247042 	 0 	 0
1 	 0 	 0 	 4.745052855503306
1 	 2.857738033247042 	 0 	 0
0 	 2.857738033247042 	 0 	 0
0 	 0 	 0 	 0
0 	 2.857738033247042 	 0 	 0
0 	 2.857738033247042 	 0 	 0
0 	 2.857738033247042 	 0 	 0
1 	 2.857738033247042 	 0 	 0
0 	 2.857738033247042 	 0 	 0
0 	 2.857738033247042 	 0 	 0
0 	 2.857738033247042 	 0 	 0
1 	 2.857738033247042 	 0 	 0
1 	 2.857738033247042 	 0 	 0
1 	 2.857738033247042 	 0 	 0
0 	 2.857738033247042 	 0 	 0
1 	 2.857738033247042 	 0 	 0
0 	 2.857738033247042 	 0 	 0
1 	 2.857738033247042 	 0 	 0
1 	 2.857738033247042 	 2.52078447201548 	 0
0 	 0 	 0 	 0
1 	 2.857738033247042 	 0 	 0
0 	 2.857738033247042 	 0 	 0
1 	 2.857738033247042 	 2.52078447201548 	 0
0

In [ ]:
# Load and parse the data
def parsePoint(line):
    values = [float(x) for x in line.split(' ')]
    return LabeledPoint(values[0], values[1:])

data = sc.textFile(r"C\spark\mllib\sample_svm_data.txt")
parsedData = data.map(parsePoint)

# Build the model
model = SVMWithSGD.train(parsedData, iterations=100)

# Evaluating the model on training data
labelsAndPreds = parsedData.map(lambda p: (p.label, model.predict(p.features)))
trainErr = labelsAndPreds.filter(lambda lp: lp[0] != lp[1]).count() / float(parsedData.count())
print("Training Error = " + str(trainErr))

# Save and load model
model.save(sc, "target/tmp/pythonSVMWithSGDModel")
sameModel = SVMModel.load(sc, "target/tmp/pythonSVMWithSGDModel")

In [ ]:
rows = iris.map(lambda line:line.split(","))
irisdatatrain = rows.map(lambda iris:labeledpoint(iris[4],iris[:3]))
#spliiting the data
iristrain, iristest = randomsplit(0.75,0.25)
model = logisticregressionithlbfgs.train(iristrain)
model.predict(iristest)

In [ ]:
#to determne the number or rows
iris = sc.textFile(r'C:\Users\jeffnerd\Desktop\IRIS.csv')
rows = iris.map(lambda line:line.split(","))
for row in rows.take(rows.count()):
    print(row[0:])

In [7]:
data = sc.textFile(r"C:\spark\mllib\sample_svm_data.txt")

In [11]:
def parsePoint(line):
    values = [float(x) for x in line.split(' ')]
    return LabeledPoint(values[0], values[1:])

data = sc.textFile(r"C:\spark\data\mllib\sample_svm_data.txt")
parsedData = data.map(parsePoint)

In [12]:
 #Build the model
model = SVMWithSGD.train(parsedData, iterations=100)

In [13]:
# Evaluating the model on training data
labelsAndPreds = parsedData.map(lambda p: (p.label, model.predict(p.features)))
trainErr = labelsAndPreds.filter(lambda lp: lp[0] != lp[1]).count() / float(parsedData.count())
print("Training Error = " + str(trainErr))

Training Error = 0.38198757763975155


In [21]:
#Logistic regression model
def parsePoint(line):
    values = [float(x) for x in line.split(' ')]
    return LabeledPoint(values[0], values[1:])

data = sc.textFile(r"C:\spark\data\mllib\sample_svm_data.txt")
parsedData = data.map(parsePoint)

# Build the model
model = LogisticRegressionWithLBFGS.train(parsedData, iterations=100)

# Evaluating the model on training data
labelsAndPreds = parsedData.map(lambda p: (p.label, model.predict(p.features)))
trainErr = labelsAndPreds.filter(lambda lp: lp[0] != lp[1]).count() / float(parsedData.count())
print("Training Error = " + str(trainErr))


Training Error = 0.36645962732919257


In [24]:
print(labelsAndPreds.collect())

[(1.0, 1), (0.0, 1), (0.0, 0), (1.0, 1), (1.0, 0), (0.0, 1), (1.0, 1), (1.0, 1), (0.0, 0), (0.0, 0), (1.0, 1), (1.0, 0), (1.0, 0), (1.0, 0), (0.0, 0), (0.0, 1), (0.0, 0), (0.0, 1), (0.0, 0), (1.0, 1), (0.0, 1), (0.0, 0), (0.0, 0), (1.0, 0), (1.0, 1), (1.0, 0), (0.0, 1), (1.0, 1), (0.0, 1), (1.0, 1), (1.0, 1), (0.0, 1), (1.0, 0), (0.0, 0), (1.0, 1), (0.0, 1), (1.0, 1), (1.0, 0), (1.0, 1), (0.0, 1), (1.0, 1), (0.0, 0), (0.0, 1), (1.0, 1), (1.0, 1), (0.0, 1), (1.0, 1), (0.0, 0), (1.0, 1), (1.0, 1), (0.0, 0), (0.0, 0), (0.0, 0), (1.0, 0), (1.0, 1), (0.0, 1), (0.0, 1), (0.0, 0), (1.0, 0), (1.0, 1), (0.0, 0), (0.0, 1), (1.0, 1), (1.0, 1), (1.0, 1), (0.0, 0), (0.0, 1), (0.0, 1), (1.0, 0), (1.0, 1), (0.0, 0), (0.0, 0), (0.0, 0), (0.0, 1), (1.0, 1), (1.0, 1), (1.0, 1), (1.0, 1), (1.0, 1), (0.0, 0), (1.0, 1), (0.0, 1), (1.0, 0), (1.0, 0), (1.0, 1), (0.0, 0), (1.0, 1), (1.0, 1), (0.0, 0), (1.0, 1), (1.0, 1), (0.0, 1), (0.0, 0), (1.0, 0), (0.0, 1), (1.0, 0), (0.0, 0), (1.0, 1), (1.0, 1), (1.0, 1),

In [25]:
parsedData.count()

322

In [26]:
labelsAndPreds.collect()

[(1.0, 1),
 (0.0, 1),
 (0.0, 0),
 (1.0, 1),
 (1.0, 0),
 (0.0, 1),
 (1.0, 1),
 (1.0, 1),
 (0.0, 0),
 (0.0, 0),
 (1.0, 1),
 (1.0, 0),
 (1.0, 0),
 (1.0, 0),
 (0.0, 0),
 (0.0, 1),
 (0.0, 0),
 (0.0, 1),
 (0.0, 0),
 (1.0, 1),
 (0.0, 1),
 (0.0, 0),
 (0.0, 0),
 (1.0, 0),
 (1.0, 1),
 (1.0, 0),
 (0.0, 1),
 (1.0, 1),
 (0.0, 1),
 (1.0, 1),
 (1.0, 1),
 (0.0, 1),
 (1.0, 0),
 (0.0, 0),
 (1.0, 1),
 (0.0, 1),
 (1.0, 1),
 (1.0, 0),
 (1.0, 1),
 (0.0, 1),
 (1.0, 1),
 (0.0, 0),
 (0.0, 1),
 (1.0, 1),
 (1.0, 1),
 (0.0, 1),
 (1.0, 1),
 (0.0, 0),
 (1.0, 1),
 (1.0, 1),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (1.0, 0),
 (1.0, 1),
 (0.0, 1),
 (0.0, 1),
 (0.0, 0),
 (1.0, 0),
 (1.0, 1),
 (0.0, 0),
 (0.0, 1),
 (1.0, 1),
 (1.0, 1),
 (1.0, 1),
 (0.0, 0),
 (0.0, 1),
 (0.0, 1),
 (1.0, 0),
 (1.0, 1),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 1),
 (1.0, 1),
 (1.0, 1),
 (1.0, 1),
 (1.0, 1),
 (1.0, 1),
 (0.0, 0),
 (1.0, 1),
 (0.0, 1),
 (1.0, 0),
 (1.0, 0),
 (1.0, 1),
 (0.0, 0),
 (1.0, 1),
 (1.0, 1),
 (0.0, 0),
 (1.0, 1),
 (1.0, 1),

In [4]:
Vectors.sparse(4, [0,2], [1.0,2.0])

SparseVector(4, {0: 1.0, 2: 2.0})